In [37]:
import json
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import rank_bm25 as bm25
import pandas as pd
import numpy as np
import re
import string
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('wordnet')
import xmltodict

[nltk_data] Downloading package wordnet to /home/mimig/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
wd = '/media/sf_year2/DD2430-Project/Podcast-Segment-Retrieval-Spotify/'
os.chdir(wd)
files = os.listdir('./data/training_episodes')
files = [f for f in files if f != 'place_holder']
#files

In [44]:
def strip_punctuation(text):
    return text.translate(text.maketrans("", "", string.punctuation))

def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    t = text.split(" ")
    res = [lemmatizer.lemmatize(w) for w in t]
    
    return " ".join(res)

def stem(text):
    stemmer = PorterStemmer()
    t = text.split(" ")
    res = [stemmer.stem(w) for w in t]
    
    return " ".join(res)

def remove_numbers(text):
    t = re.sub(r'\d+', "", text)

    return t

def clean_text(text):
    t = text.lower()
    t = strip_punctuation(t)
    t = remove_stopwords(t)
    t = remove_numbers(t)
    # t = stem(t)
    t = lemmatize(t)
    return t
    
# Simon's  extraction function
def extract_segments(path):
    """Given path to json file containing an episode extracts all segments of that episode, 
    including start and end time of each segment."""
    with open(path, "r") as read_file:
        episode = json.load(read_file)
    segments=[]
    segment_transcripts = []
    clean_segment_transcripts = []
    #had to do "manual" iteration due to irregularities in data
    iter=0
    for segment in episode["results"]:
        seg_result={}
        
        #make sure there is only one dict in this list (should be true according to dataset description)
        assert len(segment["alternatives"])==1
        segment_dict=segment["alternatives"][0]
        #sometimes "alternatives" dict is empty...
        if "words" and "transcript"  in segment_dict:
            #add segment number
            seg_result["segNum"]=iter
            #add timestamp of the first word in this segment
            seg_result["startTime"]=segment_dict["words"][0]["startTime"]
            #add timestamp of the last word in this segment
            seg_result["endTime"]=segment_dict["words"][-1]["endTime"]
            #add transcript of this segment 
            tr = segment_dict["transcript"]
            seg_result["transcript_clean"] = clean_text(tr)
            seg_result["transcript"]=tr
            segment_transcripts.append(tr)
            clean_segment_transcripts.append(seg_result["transcript_clean"])
            segments.append(seg_result)
            iter+=1
            
    return {'segments': segments, "full_text": "".join(segment_transcripts), "full_text_clean": "".join(clean_segment_transcripts),}


In [45]:
# all_segments = []
# all_starts = []
# all_ends = []
# corpus = []
# for filename in files:
#     with open(os.path.join('./data', filename), 'rb') as f:
#         dat = json.load(f)

#     segments = []
#     starts = []
#     ends = []
#     for alt in dat['results']:
#         if 'transcript' in alt['alternatives'][0].keys():
#             seg = alt['alternatives'][0]['transcript']
#             segments.append(seg)
#             s = alt['alternatives'][0]['words'][0]['startTime']
#             starts.append(s)
#             e = alt['alternatives'][0]['words'][-1]['endTime']
#             ends.append(e)

#     text = "".join(segments)
#     all_segments.append(segments)
#     all_starts.append(starts)
#     all_ends.append(ends)
#     corpus.append(text)

In [46]:
# training_sub = {}

# for filename in files: 
#     training_sub[os.path.splitext(filename)[0]] = extract_segments(os.path.join("./data/training_episodes", filename))

# with open("./data/training_sub_cleaned.json", "w") as f:
#     json.dump(training_sub, f)

with open("./data/training_sub_cleaned.json", "r") as f:
    training_sub = json.load(f)

In [87]:
episodes = list(training_sub.keys())
full_texts_clean = [training_sub[k]['full_text_clean']for k in list(training_sub.keys())]
full_texts = [training_sub[k]['full_text']for k in list(training_sub.keys())]

In [48]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df=0.9, stop_words={'english'})
X = vectorizer.fit_transform(full_texts_clean)
tf_idf_df = pd.DataFrame(
    X.toarray(),
    columns = vectorizer.get_feature_names()
)
# tf_idf_df

In [54]:
with open(os.path.join("./data", "podcasts_2020_topics_train.xml"), "r") as f:
    topics = f.readlines()
    
topics = "".join(topics)  
topics = xmltodict.parse(topics)

temp = {}
for t in topics['topics']['topic']:
    n = int(t['num'])
    temp[n] = {}
    temp[n]['query'] = t['query']
#     temp[n]['query_clean'] = clean_text(t['query'])
    temp[n]['type'] = t['type']
    temp[n]['description'] = t['description']
    temp[n]['description_clean'] = clean_text(t['description'])
    
topics = temp
# topics

In [80]:
topic = topics[1]["description_clean"]
tokens = [x for x in topic.split(" ") if x not in ["", " "]]

bigrams = []
for gram in nltk.ngrams(tokens, 2):
    bigrams.append(" ".join(gram))

    
tokens.extend(bigrams)
# tokens

In [79]:
score = np.zeros((tf_idf_df.shape[0]))
for t in tokens:
    if t in tf_idf_df.columns:
        score = score + tf_idf_df.loc[:, t]
        
top_episodes = np.argsort(-score)[0:10]

In [89]:
print(topic)
for i in top_episodes:
    print(episodes[i])
    print(full_texts[i][:1000])

people saying spread novel coronavirus ncov wuhan end 
5MM6w9RTRYj5ePligGvZNs
You ready? If you're ready. Yes, you'll get a shout-out to Almighty supers out there. We are gathered here today to substitute eat, honey. So make sure you guys have your tea cups ready because you already know this T is what - Hi. How are you guys? Good afternoon. I hope everybody's doing good today. So I want to come on here with another podcast. So this is going to be very different from the typical videos that I usually do and this goes back to two days ago right before Kobe Bryant's death. So if you follow me on Instagram which unfortunately My page is private because of people trolling and being disrespectful. So it's bad private again before Kobe Bryant died. I was sent I was sent pictures of this new delicacy. That's all the rage right now in China, which is called bat soup, and it's been a delicacy for a while now and so me and one of my subscribers we're talking about this and she's like you got to 

In [10]:
# flatten = lambda l: [item for sublist in l for item in sublist]
# flat_segments = flatten(all_segments)
# tokenized_segments = [x.split(" ") for x in flat_segments]
# tokenized_segments

In [11]:
bm25 = BM25Okapi(tokenized_segments)

query = "climate change strike"
tokenized_query = query.split(" ")

bm25.get_top_n(tokenized_query, flat_segments, n=2)

NameError: name 'BM25Okapi' is not defined